In [25]:
from rasa_nlu.training_data import load_data
from rasa_nlu.model import Trainer
from rasa_nlu import config

In [16]:
training_data = load_data('RASA_restaurant_data.json')
trainer = Trainer(config.load("config.yml"))

#trainer = Trainer(config.load("supervised_embedding_config.yml"))
#config_file = {"pipeline": "spacy_sklearn"}
#trainer = Trainer(config.load(config_file))

trainer.train(training_data)
model_directory = trainer.persist('./projects/default/')

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    7.4s finished
D:\Users\618757\Anaconda3\envs\rasaenv\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [21]:
print(training_data.training_examples[22])

In [17]:
from rasa_nlu.model import Interpreter

# where model_directory points to the model folder
interpreter = Interpreter.load(model_directory)
interpreter.parse(u"Looking for indian restaurant")

{'intent': {'name': 'restaurant_search', 'confidence': 0.9978131254703853},
 'entities': [{'start': 12,
   'end': 18,
   'value': 'indian',
   'entity': 'cuisine',
   'confidence': 0.7260083509825286,
   'extractor': 'CRFEntityExtractor'}],
 'intent_ranking': [{'name': 'restaurant_search',
   'confidence': 0.9978131254703853},
  {'name': 'send_mail', 'confidence': 0.0018068804349220247},
  {'name': 'thank', 'confidence': 0.00030894242742057315},
  {'name': 'affirm', 'confidence': 2.655607568568687e-05},
  {'name': 'greet', 'confidence': 2.0473828518278502e-05},
  {'name': 'bye', 'confidence': 1.5745627010834664e-05},
  {'name': 'deny', 'confidence': 8.276136057094383e-06}],
 'text': 'Looking for indian restaurant'}

In [ ]:
# this block is how to write sql query to get the rsults from SQL DB based on RASA NLU Model
message = "a cheap hotel in the north"
data = interpreter.parse(message)

'''
output of data:
{'entities': [{'end': '7', 'entity': 'price', 'start': 2, 'value': 'lo'},
  {'end': 26, 'entity': 'location', 'start': 21, 'value': 'north'}],
 'intent': {'confidence': 0.9, 'name': 'hotel_search'}}
'''

params = {}
for ent in data["entities"]:
    params[ent["entity"]] = ent["value"]

# output of params -> {'location': 'north', 'price': 'lo'}
    

query = "select name FROM hotels"

filters = ["{}=?".format(k) for k in params.keys()]
# output of filters -> ['price=?', 'location=?']

conditions = " and ".join(filters)
# output of conditions -> 'price=? and location=?'

final_q = " WHERE ".join([query, conditions])
# output of final_q ->  'SELECT name FROM hotels WHERE price=? and location=?'

t = tuple(params)


In [ ]:
import sqlite3
conn = sqlite3.connect("hotels.db")
conn.execute(final_q, t)

In [ ]:
INIT = 0
CHOOSE_COFFEE = 1
ORDERED = 2

policy_rules = {
    (INIT, "order"): (CHOOSE_COFFEE, "ok, Columbian or Kenyan?"),
    (CHOOSE_COFFEE, "specify_coffee"): 
    (ORDERED, "perfect, the beans are on their way!"),
}

policy_rules = {
    (Start, "Start"): (INIT, "I'm a bot to help you order coffee beans"),
    (INIT, "order"): (CHOOSE_COFFEE, "ok, Colombian or Kenyan?"),
    (____, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!"),
    (____, "____"): (CHOOSE_COFFEE, "We have two kinds of coffee beans - the Kenyan ones make a slightly sweeter coffee, and cost $6. The Brazilian beans make a nutty coffee and cost $5.")    
}

messages = [
    "I'd like to become a professional dancer",
    "well then I'd like to order some coffee",
    "my favourite animal is a zebra",
    "kenyan"
]

state = INIT

def interpret(message):
    data = interpreter.parse(message)
    

def respond(state, message):
   (new_state, response) = policy_rules[(state, interpret(message))]
   return new_state, response

def send_message(state, message):
    new_state, response = respond(state, message)
    return new_state

for message in messages:    
    state = send_message(state, message)